# MA_DCA Strategy Backtest

这个notebook用于对多个交易对进行MeanReverter策略的批量优化。

In [ ]:
import os
import pandas as pd
from datetime import datetime
import backtrader as bt
import quantstats as qs
import numpy as np
import btplotting 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, message="`product` is deprecated")

In [2]:
from MA_DCA import MA_DCA

# CONFIG 配置保持在文件最开始
CONFIG = {
    # 策略相关配置
    'strategy': {
        'class': MA_DCA,
        'name': MA_DCA.__name__
    },
    
    # 数据相关配置
    'symbol': 'RENDERUSDT',
    'data_path': r'\\znas\Main\futures',
    'start_date': '2024-01-01',
    'end_date': '2025-03-18',
    'source_timeframe': '1m',
    'target_timeframes': '15min',
    
    # 交易对映射关系适用于交易对有改名字的情况
    'symbol_mapping': {
        'RENDERUSDT': 'RDNTUSDT'
    },
    
    # 文件保存配置
    'reports_path': 'reports',
    
    # 回测参数配置
    'commission': 0.0004,
    'initial_capital': 10000,
    'trade_on_close': True,
    'exclusive_orders': True,
    'hedging': False,
}

In [3]:
def get_timeframe_params(timeframe_str):
    """
    将时间周期字符串转换为 backtrader 的 timeframe 和 compression 参数
    """
    if timeframe_str.endswith('min'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('min', '')))
    elif timeframe_str.endswith('H'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('H', '')) * 60)
    elif timeframe_str.endswith('D'):
        return (bt.TimeFrame.Days, 1)
    elif timeframe_str == '1m':
        return (bt.TimeFrame.Minutes, 1)
    else:
        raise ValueError(f"不支持的时间周期格式: {timeframe_str}")

def load_and_resample_data(symbol, start_date, end_date, source_timeframe='1m', target_timeframe='30min', data_path=r'\\znas\Main\futures'):
    """
    加载并重采样期货数据
    """
    # 生成日期范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 准备存储所有数据的列表
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 使用CONFIG中的symbol_mapping进行符号映射
    file_symbol = CONFIG['symbol_mapping'].get(formatted_symbol, formatted_symbol)
    
    # 遍历每一天
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径，使用映射后的符号
        file_path = os.path.join(data_path, date_str, f"{date_str}_{file_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并所有数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    
    # 设置时间索引
    combined_df.set_index('datetime', inplace=True)
    
    # 重采样到目标时间周期
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    # 转换为backtesting.py格式
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    
    # 最终清理
    backtesting_df = backtesting_df.dropna()
    
    # 使用辅助函数转换字符串时间周期为 timeframe 与 compression 参数
    timeframe, compression = get_timeframe_params(target_timeframe)
    
    # 注意这里构造的是 bt.feeds.PandasData 对象，而不是直接返回 DataFrame
    data_feed = bt.feeds.PandasData(
        dataname=backtesting_df,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date),
        todate=pd.to_datetime(end_date)
    )
    
    return data_feed

In [4]:
def run_backtest():
    # 从 CONFIG 中取出必要的参数（只回测单一交易对和单一时间周期）
    symbol = CONFIG['symbol']
    start_date = CONFIG['start_date']
    end_date = CONFIG['end_date']
    source_timeframe = CONFIG['source_timeframe']
    # 固定目标时间周期为 '30min'
    target_timeframe = CONFIG['target_timeframes']
    data_path = CONFIG['data_path']
    
    initial_capital = CONFIG['initial_capital']
    commission = CONFIG['commission']
    
    # 数据加载与预处理，确保返回值为 bt.feeds.PandasData 实例
    data = load_and_resample_data(
        symbol=symbol,
        start_date=start_date,
        end_date=end_date,
        source_timeframe=source_timeframe,
        target_timeframe=target_timeframe,
        data_path=data_path
    )
    
    # 检查 data_feed 的底层 DataFrame 数据
    # print("检查 data_feed 内部底层 DataFrame 信息:")
    # df_underlying = data.p.dataname  # 获取 PandasData 内部保存的 DataFrame
    # print("前5行数据:")
    # print(df_underlying.head())
    # print("\n数据信息:")
    # print(df_underlying.info())
    # print("\n每列的空值数量:")
    # print(df_underlying.isnull().sum())
    
    cerebro = bt.Cerebro()
    cerebro.adddata(data)
    cerebro.addstrategy(CONFIG['strategy']['class'])
    
    # 记录初始资金
    initial_capital = CONFIG['initial_capital']
    cerebro.broker.setcash(initial_capital)
    cerebro.broker.setcommission(commission=CONFIG['commission'])
    
    # 添加分析器
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')  # 添加PyFolio分析器
    
    # print("初始资金:", initial_capital)
    strategies = cerebro.run()
    strat = strategies[0]
    final_value = cerebro.broker.getvalue()
    # print("结束资金:", final_value)
    
    # 计算基本回测指标
    profit = final_value - initial_capital
    roi = (profit / initial_capital) * 100
    
    print("\n=== 基本回测指标 ===")
    print(f"起始资金: {initial_capital:.2f}")
    print(f"最终资金: {final_value:.2f}")
    print(f"总利润: {profit:.2f}")
    print(f"ROI: {roi:.2f}%")
    
    # 获取PyFolio的分析结果并处理返回数据
    portfolio_stats = strat.analyzers.pyfolio.get_pf_items()
    returns, positions, transactions, gross_lev = portfolio_stats
    returns.index = returns.index.tz_convert(None)  # 移除时区信息
    
    # 在生成报告之前，确保reports目录存在
    reports_dir = 'reports'
    if not os.path.exists(reports_dir):
        os.makedirs(reports_dir)
        print(f"创建报告目录: {reports_dir}")
    
    # 生成QuantStats报告
    report_path = f'reports/{CONFIG["strategy"]["name"]}_{CONFIG["symbol"]}_{CONFIG["start_date"]}_{CONFIG["end_date"]}.html'
    report_title = f'{CONFIG["strategy"]["name"]} Strategy Analysis - {CONFIG["symbol"]}'
    
    try:
        # 处理警告
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)
        
        # 确保returns是正确的格式
        if isinstance(returns, pd.Series):
            returns = returns.copy()  # 创建副本避免修改原始数据
            # 确保索引是datetime格式
            returns.index = pd.to_datetime(returns.index)
        
        qs.reports.html(
            returns=returns,
            output=report_path,
            title=report_title,
            benchmark=None,  # 可以添加基准收益进行对比
            strategy_name=CONFIG['strategy']['name'],
            trades=transactions
        )
        print(f"\nQuantStats报告已生成: {report_path}")
        
        # 输出详细的策略指标
        print("\n=== 详细策略指标 ===")
        try:
            print(f"Sharpe Ratio: {qs.stats.sharpe(returns):.2f}")
            print(f"Sortino Ratio: {qs.stats.sortino(returns):.2f}")
            print(f"Calmar Ratio: {qs.stats.calmar(returns):.2f}")
            print(f"Max Drawdown: {qs.stats.max_drawdown(returns):.2%}")
            print(f"Win Rate: {qs.stats.win_rate(returns):.2%}")
            print(f"Profit Factor: {qs.stats.profit_factor(returns):.2f}")
            print(f"Expected Return (月): {qs.stats.expected_return(returns, aggregate='M'):.2%}")
            print(f"Kelly Criterion: {qs.stats.kelly_criterion(returns):.2%}")
            print(f"Risk of Ruin: {qs.stats.risk_of_ruin(returns):.2%}")
            print(f"Tail Ratio: {qs.stats.tail_ratio(returns):.2f}")
            print(f"Common Sense Ratio: {qs.stats.common_sense_ratio(returns):.2f}")
            
            # 添加更多详细指标
            print("\n=== 额外策略指标 ===")
            print(f"Average Win: {qs.stats.avg_win(returns):.2%}")
            print(f"Average Loss: {qs.stats.avg_loss(returns):.2%}")
            print(f"Volatility (年化): {qs.stats.volatility(returns, periods=252):.2%}")
            print(f"Skew: {qs.stats.skew(returns):.2f}")
            print(f"Kurtosis: {qs.stats.kurtosis(returns):.2f}")
            print(f"Value at Risk: {qs.stats.value_at_risk(returns):.2%}")
            print(f"Conditional VaR: {qs.stats.conditional_value_at_risk(returns):.2%}")
            print(f"Payoff Ratio: {qs.stats.payoff_ratio(returns):.2f}")
            print(f"Gain to Pain Ratio: {qs.stats.gain_to_pain_ratio(returns):.2f}")
            print(f"Ulcer Index: {qs.stats.ulcer_index(returns):.2f}")
            
            # 连续盈亏统计
            print("\n=== 连续盈亏统计 ===")
            print(f"最长连续盈利次数: {qs.stats.consecutive_wins(returns)}")
            print(f"最长连续亏损次数: {qs.stats.consecutive_losses(returns)}")
            
        except Exception as e:
            print(f"计算某些指标时出错: {str(e)}")
        
    except Exception as e:
        print(f"生成QuantStats报告时出错: {str(e)}")
    
    # 绘制回测图形
    plot = btplotting.BacktraderPlotting()
    cerebro.plot(plot)  # iplot=False 使用静态图表而不是交互式图表


In [ ]:
if __name__ == '__main__':
    run_backtest()